<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp-phase1/blob/main/course4/week1/Convolution_model_Step_by_Step_v2a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Networks: Step by Step #

4번째 코스의 첫 과제에 오신것을 환영합니다! 이번 과제에서 numpy를 활용해 여러분은 합성곱 신경망(CONV)와 풀링(POOL) 레이어의 정방향 연산과, 역전파를 구현해봅니다.

**표기법**:
- 위첨자 $[l]$은 $l$ 번째 레이어에 있는 객체라는 것을 의미합니다.
  - 예를 들어, $a^{[4]}$는 4번째 층의 활성화 변수이며, $W^{[5]}$와 $b^{[5]}$는 5번째 층의 파라미터를 의미합니다.
- 위첨자 $(i)$는 데이터 세트의 $i$번째 데이터를 의미합니다.
  - 예를 들어 $x^{(i)}$는 훈련 세트 x의 i번째 데이터를 의미합니다.
- 아래첨자 $i$는 벡터의 $i$번 째 아이템을 의미합니다.
  - 예를 들어 $a^{[l]}_i$는 l번째 레이어(이 레이어는 Fully Connected 레이어라고 가정합니다)의 활성화 벡터 중 i번째 값을 의미합니다.
- $n_H$, $n_W$, $n_C$는 각각 주어진 층에서의 높이, 너비, 채널의 개수를 의미합니다. 만약 명시적으로 층을 표기하고 싶다면, $n_H^{[l]}$, $n_W^{[l]}$, $n_C^{[l]}$과 같은 방법으로 표기합니다.
- $n_{H_{prev}}$, $n_{W_{prev}}$, $n_{C_{prev}}$는 이전 층의 높이, 너비, 채널의 개수를 의미합니다. 특정한 레이어 $l$에 대해서 표현하고 싶다면, $n_H^{[l-1]}$, $n_W^{[l-1]}$, $n_C^{[l-1]}$과 같이 표현합니다.

지난 과제들을 수행하면서 여러분들은 이미 `numpy` 라이브러리에 익숙해졌을 것입니다. 그렇다고 가정하고, 이번 과제를 시작해보겠습니다.

## 1. Packages ##

과제 수행을 위해 필요한 패키지를 받아보겠습니다.

- [numpy](www.numpy.org)는 과학 연산을 위해 필요한 가장 기본적인 파이썬 패키지입니다.
- [matplotlib](http://matplotlib.org)은 그래프를 그리기 위한 파이썬 라이브러리입니다.
- `np.random.seed(1)` 코드는 랜덤 함수가 일관된 값을 리턴하도록 합니다. 여러분의 과제를 채점하는데 필요하니 지우거나 변경하지 마세요.

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

## 2. Outline of the Assignment ##

지금부터 여러분은 컨볼루션(합성곱) 신경망의 블록들을 구현해볼 것입니다. 구현해야할 개별 함수들은 구체적인 지시사항이 명시되어 있으므로 그 지시사항대로 구현해보세요. 전반적인 구현 방향은 아래와 같습니다.
- 컨볼루션 레이어 함수(Convolution functions)는 아래 4개 함수를 포함합니다:
  - Zero Padding
  - Convolve Window
  - Convolution forward
  - Convolution backward(optional)
- 풀링 레이어 함수(Pooling functions)는 아래 4개 함수를 포함합니다:
  - Pooling forward
  - Create mask
  - Distribute value
  - Pooling backward(optinal)


이 과제는 `numpy`를 사용해 이러한 기능을 처음부터 구현하도록 요청합니다. 다음 과제부터는 이러한 함수와 동등한 기능을 제공하는 TensorFlow 빌트인 함수를 사용하여 모델을 빌드합니다.

<img src="arts/model.png" style="width:800px;height:300px;">


**참고** 모든 정방향 함수에는 매칭되는 역방향 함수가 있습니다. 따라서 정방향 연산 모듈의 모든 단계에서 일부 매개 변수를 캐시에 저장합니다. 이러한 매개 변수는 역 전파 중에 기울기를 계산하는 데 사용됩니다.

## 3. Convolutional Neural Networks ##

다양한 딥 러닝 프레임워크들이 컨볼루션 연산을 사용하기 쉽게 제공해주지만, 사실 이 연산은 딥 러닝에서 가장 이해하기 어려운 개념 중 하나입니다. 컨볼루션 신경망은 3차원의 입력 볼륨(Volume)을 다른 사이즈의 출력 volume으로 변환시켜줍니다. 아래 그림처럼 말이죠.

<img src="arts/conv_nn.png" style="width:350px;height:200px;">

이 파트에서, 여러분은 컨볼루션 레이어의 모든 함수를 단계별루 구현해봅니다. 처음으로 두 개의 helper 함수를 만들어 보겠습니다. zero padding과 convolution 연산입니다.


### 3-1. Zero-Padding ###

Zero-padding 연산은 이미지의 양 가장자리에 0으로 된 테두리를 집어넣는 연산입니다. 아래 그림을 확인해보세요.

<img src="arts/PAD.png" style="width:600px;height:400px;">

<center><b>그림 1</b> : Zero-Padding</center>
<center>Image(RGB 3채널, 사이즈 2의 패딩)</center>

<br>

Padding을 사용했을 때의 주된 이점은 다음과 같습니다:
- input 볼륨의 높이와 너비를 축소하지 않아도 컨볼루션 레이어를 사용할 수 있습니다. 이것은 더 깊은 컨볼루션 신경망을 구축하는데 중요합니다. padding을 사용하지 않으면 더 깊은 레이어로 들어갈수록 높이/너비가 줄어들기 때문입니다. 기억해야할 케이스는 "동일한" 컨볼루션으로, input과 output의 높이/너비가 일치하는 경우입니다.
- 이미지 가장자리의 데이터를 유지하는데 도움이 됩니다. 패딩이 없다면 output레이어의 일부만이 이미지 가장자리 픽셀의 영향을 받습니다.

**연습 문제**: 배치만큼의 주어진 이미지에 패딩을 추가하는 아래 함수를 구현해보세요. [Use np.pad](https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html) 함수를 사용하세요. (5, 5, 5, 5, 5)의 shape를 가지고 있는 배열 `a`에 대해, 2번째 차원에 `pad = 1`, 3번째 차원에 `pad = 3`, 4번째 차원에 `pad=0`을 하고 싶다면, 아래 코드처럼 작성하면 됩니다.
```python
a = np.pad(a, ((0,0), (1,1), (0,0), (3,3), (0,0)), mode = 'constant', constant_values = (0,0))
```


In [ ]:
# GRADED FUNCTION: zero_pad

def zero_pad(X, pad):
    """
    Pad with zeros all images of the dataset X. The padding is applied to the height and width of an image, 
    as illustrated in Figure 1.
    
    Argument:
    X -- python numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
    pad -- integer, amount of padding around each image on vertical and horizontal dimensions
    
    Returns:
    X_pad -- padded image of shape (m, n_H + 2*pad, n_W + 2*pad, n_C)
    """
    
    ### START CODE HERE ### (≈ 1 line)
    print(X, ())
    X_pad = None
    ### END CODE HERE ###
    
    return X_pad

In [ ]:
np.random.seed(1)
x = np.random.randn(4, 3, 3, 2)
x_pad = zero_pad(x, 2)
print ("x.shape =\n", x.shape)
print ("x_pad.shape =\n", x_pad.shape)
print ("x[1,1] =\n", x[1,1])
print ("x_pad[1,1] =\n", x_pad[1,1])

fig, axarr = plt.subplots(1, 2)
axarr[0].set_title('x')
axarr[0].imshow(x[0,:,:,0])
axarr[1].set_title('x_pad')
axarr[1].imshow(x_pad[0,:,:,0])

**모범 답안**:
```
x.shape =
 (4, 3, 3, 2)
x_pad.shape =
 (4, 7, 7, 2)
x[1,1] =
 [[ 0.90085595 -0.68372786]
 [-0.12289023 -0.93576943]
 [-0.26788808  0.53035547]]
x_pad[1,1] =
 [[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]
```

### 3-2. Single step of convolution ###

이 부분에서는 입력 volume의 고정된 위치에 필터를 적용하는 단일 컨볼루션 단계를 구현합니다. 이것은 다음과 같은 컨볼 루션 단위를 만드는 데 사용됩니다.

- 입력 volume을 가지고
- 모든 position에 대해 필터를 적용한 이후
- 합성곱이 계산된 출력 volume을 리턴합니다 (이 volume의 size는 입력과는 다를 가능성이 높습니다)

<img src="arts/Convolution_schematic.gif" style="width:500px;height:300px;">

<center><b>Figure 2</b>: 컨볼루션 연산</center>
<center>3x3 사이즈의 필터에, stride(필터를 얼마만큼 넘길건지에 대한 값) = 1로 적용한 예시입니다</center>


컴퓨터 비전 프로그램에서 그림의 왼쪽에 있는 행렬의 각 값은 단일 픽셀 값에 해당하며, 이 값을 원래 행렬과 요소별로 곱(element-wise)한 다음 더하고 bias를  추가하여 이미지와 3x3 필터를 컨볼 루션합니다. 이 연습 문제의 첫 번째 단계에서는 단일 실수 값 출력을 얻기 위해 여러 position 중 주어진 단일 position에만 필터를 적용하는 단일 회선 단계를 구현합니다.


과제 뒷부분에서 이 함수를 입력의 여러 위치에 적용하여 전체 컨벌루션 연산을 구현할 것입니다.

**연습 문제**: `conv_single_step()`함수를 구현하세요. [Hint](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.sum.html)

**주의** : 
변수 `b`는 numpy 배열로 전달됩니다. numpy 배열에 스칼라 (float 또는 integer)를 추가하면 결과는 numpy 배열입니다. numpy 배열에 단일 값이 포함 된 특별한 경우에는 이를 부동 소수점으로 캐스팅하여 스칼라로 변환 할 수 있습니다.

In [ ]:
# GRADED FUNCTION: conv_single_step

def conv_single_step(a_slice_prev, W, b):
    """
    Apply one filter defined by parameters W on a single slice (a_slice_prev) of the output activation 
    of the previous layer.
    
    Arguments:
    a_slice_prev -- slice of input data of shape (f, f, n_C_prev)
    W -- Weight parameters contained in a window - matrix of shape (f, f, n_C_prev)
    b -- Bias parameters contained in a window - matrix of shape (1, 1, 1)
    
    Returns:
    Z -- a scalar value, the result of convolving the sliding window (W, b) on a slice x of the input data
    """

    ### START CODE HERE ### (≈ 2 lines of code)
    # Element-wise product between a_slice_prev and W. Do not add the bias yet.
    s = None
    # Sum over all entries of the volume s.
    Z = None
    # Add bias b to Z. Cast b to a float() so that Z results in a scalar value.
    Z = None
    ### END CODE HERE ###

    return Z

In [ ]:
np.random.seed(1)
a_slice_prev = np.random.randn(4, 4, 3)
W = np.random.randn(4, 4, 3)
b = np.random.randn(1, 1, 1)

Z = conv_single_step(a_slice_prev, W, b)
print("Z =", Z)

**모범 답안**:
<table>
    <tr>
        <td>
            <b>Z</b>
        </td>
        <td>
            -6.99908945068
        </td>
    </tr>
</table>

### 3-3. Convolutional Neural Networks - Forward pass ###

정방향 연산에서는 많은 필터를 사용하여 입력에 컨볼 루션합니다. 각 '컨볼루션'은 2D 매트릭스 출력을 제공합니다. 필터의 수 만큼 컨볼루션 연산을 수행한 다음 다음 출력을 쌓아 3D 볼륨을 얻습니다.

<img src="arts/conv_kiank.png" style="width:500px;height:300px;">

**연습 문제**: `W` 필터와 입력 활성화 volume인 `A_prev`를 합성곱 연산하는 아래 함수를 구현하세요. 함수는 아래와 같은 인자를 받습니다.
- `A_prev` : (1 batch: m 개의 데이터 만큼의) 이전 층의 활성화 출력값입니다.
- 가중치는 `W`로 나타냅니다. 필터의 사이즈는 `f`x`f`입니다.
- bias 벡터는 `b`입니다. 각각의 필터는 별도의 bias 값을 가지고 있습니다.

또한 여러분은 `hyperparameters` 딕셔너리를 통해서 stride와 padding 값에 접근할 수 있습니다.

**힌트**:
1. `a_prev`(`(5,5,3)`의 shape) 배열의 좌상단 2x2 슬라이스를 선택합니다.
  ```python
  a_slice_prev = a_prev[0:2, 0:2, :]
  ```
`a_slice_prev`는 3D의 단면으로, 높이 2, 너비 2, 깊이가 3인 배열입니다. 여기서 깊이는 채널의 갯수일 것입니다.
아래와 같이 `start`/`end` 인덱스를 통해 `a_slice_prev`를 동적으로 정의하는 것이 좋습니다.

2. 슬라이스를 정의하기 위해, 먼저 슬라이스의 각 꼭짓점 `vert_start`, `vert_end`, `horiz_start`, `horiz_end`를 정의합니다. 아래 그림은 코드에서 h, w, f, s를 이용해 슬라이스의 각 꼭짓점을 정의하는 방법을 찾는데 도움이 될 수 있습니다.

<img src="arts/vert_horiz_kiank.png" style="width:400px;height:300px;">

<center><b>그림 3</b>: start/end 변수를 이용해 단일 채널에서 slice 정의하기(2x2 필터에서)</center>

**리마인더**: 주어진 입력층의 shape에 대해 출력층의 shape를 구하는 공식은 다음과 같습니다.
$$ n_H = \lfloor \frac{n_{H_{prev}} - f + 2 \times pad}{stride} \rfloor +1 $$
$$ n_W = \lfloor \frac{n_{W_{prev}} - f + 2 \times pad}{stride} \rfloor +1 $$
$$ n_C = \text{number of filters used in the convolution}$$

이번 과제에서 벡터화 이슈는 크게 걱정하지 않아도 됩니다. 모든 연산을 for 문으로 처리해도 좋습니다



**애로사항이 있을 경우 추가 힌트**
- `a_prev_pad`, `W`, `b`에 대해서 array slicing(e.g. `varname[0:1, :, 3:5]`)을 사용해야 합니다.
함수의 시작 코드를 복사하고 정의 된 함수 외부에서 별도의 셀에서 실행해보세요.
각 배열의 하위 집합이 원하는 크기와 차원인지 확인합니다.

- `vert_start`, `vert_end`; `horiz_start`, `horiz_end`를 구하기 위해서 이 값이 이전 레이어의 index라는 것을 기억하세요.

- 이전에 패딩 된 레이어 (예 : 8 x 8)와 현재 (출력 레이어) (예 : 2 x 2)의 예를 그립니다.
출력 레이어의 인덱스는 h와 w로 표시됩니다.

- `a_slice_prev`에 높이, 너비 및 깊이가 있는지 확인하십시오.

- `a_prev_pad`는 `A_prev_pad`의 하위 집합입니다.
for 루프 내에서 어떤 것을 사용해야하는지 생각해보십시오.



In [ ]:
# GRADED FUNCTION: conv_forward

def conv_forward(A_prev, W, b, hparameters):
    """
    Implements the forward propagation for a convolution function
    
    Arguments:
    A_prev -- output activations of the previous layer, 
        numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    W -- Weights, numpy array of shape (f, f, n_C_prev, n_C)
    b -- Biases, numpy array of shape (1, 1, 1, n_C)
    hparameters -- python dictionary containing "stride" and "pad"
        
    Returns:
    Z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward() function
    """
    
    ### START CODE HERE ###
    # Retrieve dimensions from A_prev's shape (≈1 line)  
    (m, n_H_prev, n_W_prev, n_C_prev) = None
    
    # Retrieve dimensions from W's shape (≈1 line)
    (f, f, n_C_prev, n_C) = None
    
    # Retrieve information from "hparameters" (≈2 lines)
    stride = None
    pad = None
    
    # Compute the dimensions of the CONV output volume using the formula given above. 
    # Hint: use int() to apply the 'floor' operation. (≈2 lines)
    n_H = None
    n_W = None
    
    # Initialize the output volume Z with zeros. (≈1 line)
    Z = None
    
    # Create A_prev_pad by padding A_prev
    A_prev_pad = None
    
    for i in range(None):               # loop over the batch of training examples
        a_prev_pad = None               # Select ith training example's padded activation
        for h in range(None):           # loop over vertical axis of the output volume
            # Find the vertical start and end of the current "slice" (≈2 lines)
            vert_start = None
            vert_end = None
            
            for w in range(None):       # loop over horizontal axis of the output volume
                # Find the horizontal start and end of the current "slice" (≈2 lines)
                horiz_start = None
                horiz_end = None
                
                for c in range(None):   # loop over channels (= #filters) of the output volume
                                        
                    # Use the corners to define the (3D) slice of a_prev_pad (See Hint above the cell). (≈1 line)
                    a_slice_prev = None
                    
                    # Convolve the (3D) slice with the correct filter W and bias b, to get back one output neuron. (≈3 line)
                    weights = None
                    biases = None
                    Z[i, h, w, c] = None
                                        
    ### END CODE HERE ###
    
    # Making sure your output shape is correct
    assert(Z.shape == (m, n_H, n_W, n_C))
    
    # Save information in "cache" for the backprop
    cache = (A_prev, W, b, hparameters)
    
    return Z, cache

In [ ]:
np.random.seed(1)
A_prev = np.random.randn(10,5,7,4)
W = np.random.randn(3,3,4,8)
b = np.random.randn(1,1,1,8)
hparameters = {"pad" : 1,
               "stride": 2}

Z, cache_conv = conv_forward(A_prev, W, b, hparameters)
print("Z's mean =\n", np.mean(Z))
print("Z[3,2,1] =\n", Z[3,2,1])
print("cache_conv[0][1][2][3] =\n", cache_conv[0][1][2][3])

**모범 답안**:
```
Z's mean =
 0.692360880758
Z[3,2,1] =
 [ -1.28912231   2.27650251   6.61941931   0.95527176   8.25132576
   2.31329639  13.00689405   2.34576051]
cache_conv[0][1][2][3] = [-1.1191154   1.9560789  -0.3264995  -1.34267579]
```

마지막으로 CONV 레이어에는 활성화도 포함되어야합니다.이 경우 다음 코드 줄을 추가합니다.
```python
# Convolve the window to get back one output neuron
Z[i, h, w, c] = ...
# Apply activation
A[i, h, w, c] = activation(Z[i, h, w, c])
```

다만 이 여기서 직접적으로 추가해줄 필요는 없습니다.

## 4. Pooling layer ##

풀링(Pooling) 레이어는 입력층의 높이와 너비를 줄입니다. 이는 계산을 줄이는데 도움을 줄 뿐만 아니라, 특성을 감지할 때 입력의 위치에 영향을 받지 않도록 해줍니다. 두 가지 유형의 풀링 레이어는 다음과 같습니다.

- Max-pooling layer : ($f, f$)의 창을 입력 위에 겹쳐 슬라이드하고, 각 창의 최대값을 출력에 저장합니다.

- Average-pooling layer: ($f, f$)의 창을 입력 위에 겹쳐 슬라이드하고 각 창의 평균값을 출력에 저장합니다.

<table>
<td>
<img src="arts/max_pool1.png" style="width:500px;height:300px;">
<td>
<td>
<img src="arts/a_pool.png" style="width:500px;height:300px;">
<td>
</table>

Pooling 레이어에는 훈련할 역전파 파라미터가 없습니다. 그러나 창의 크기($f$)와 같은 하이퍼 파라미터가 있습니다. 이 값은 `max` 혹은 `average`를 계산할 $f \times f$창의 높이와 너비를 지정합니다.



### 4-1. Forward Pooling ###

지금부터 MAX-POOL과 AVG-POOL을 같은 함수 내부에서 구현해보겠습니다.

**연습 문제**: Pooling 레이어의 정방향 계산을 구현하세요. 아래 힌트를 따라서 구현하면 됩니다.

**리마인더**: Padding이 없다면, 풀링 레이어의 출력층은 아래와 같은 shape의 배열을 리턴합니다.

$$ n_H = \lfloor \frac{n_{H_{prev}} - f}{stride} \rfloor +1 $$

$$ n_W = \lfloor \frac{n_{W_{prev}} - f}{stride} \rfloor +1 $$

$$ n_C = n_{C_{prev}}$$

In [ ]:
# GRADED FUNCTION: pool_forward

def pool_forward(A_prev, hparameters, mode = "max"):
    """
    Implements the forward pass of the pooling layer
    
    Arguments:
    A_prev -- Input data, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    hparameters -- python dictionary containing "f" and "stride"
    mode -- the pooling mode you would like to use, defined as a string ("max" or "average")
    
    Returns:
    A -- output of the pool layer, a numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache used in the backward pass of the pooling layer, contains the input and hparameters 
    """
    
    # Retrieve dimensions from the input shape
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # Retrieve hyperparameters from "hparameters"
    f = hparameters["f"]
    stride = hparameters["stride"]
    
    # Define the dimensions of the output
    n_H = int(1 + (n_H_prev - f) / stride)
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev
    
    # Initialize output matrix A
    A = np.zeros((m, n_H, n_W, n_C))              
    
    ### START CODE HERE ###
    for i in range(None):                         # loop over the training examples
        for h in range(None):                     # loop on the vertical axis of the output volume
            # Find the vertical start and end of the current "slice" (≈2 lines)
            vert_start = None
            vert_end = None
            
            for w in range(None):                 # loop on the horizontal axis of the output volume
                # Find the vertical start and end of the current "slice" (≈2 lines)
                horiz_start = None
                horiz_end = None
                
                for c in range (None):            # loop over the channels of the output volume
                    
                    # Use the corners to define the current slice on the ith training example of A_prev, channel c. (≈1 line)
                    a_prev_slice = None
                    
                    # Compute the pooling operation on the slice. 
                    # Use an if statement to differentiate the modes. 
                    # Use np.max and np.mean.
                    if mode == "max":
                        A[i, h, w, c] = None
                    elif mode == "average":
                        A[i, h, w, c] = None
    
    ### END CODE HERE ###
    
    # Store the input and hparameters in "cache" for pool_backward()
    cache = (A_prev, hparameters)
    
    # Making sure your output shape is correct
    assert(A.shape == (m, n_H, n_W, n_C))
    
    return A, cache

In [ ]:
# Case 1: stride of 1
np.random.seed(1)
A_prev = np.random.randn(2, 5, 5, 3)
hparameters = {"stride" : 1, "f": 3}

A, cache = pool_forward(A_prev, hparameters)
print("mode = max")
print("A.shape = " + str(A.shape))
print("A =\n", A)
print()
A, cache = pool_forward(A_prev, hparameters, mode = "average")
print("mode = average")
print("A.shape = " + str(A.shape))
print("A =\n", A)

**모범 답안**
```
mode = max
A.shape = (2, 3, 3, 3)
A =
 [[[[ 1.74481176  0.90159072  1.65980218]
   [ 1.74481176  1.46210794  1.65980218]
   [ 1.74481176  1.6924546   1.65980218]]

  [[ 1.14472371  0.90159072  2.10025514]
   [ 1.14472371  0.90159072  1.65980218]
   [ 1.14472371  1.6924546   1.65980218]]

  [[ 1.13162939  1.51981682  2.18557541]
   [ 1.13162939  1.51981682  2.18557541]
   [ 1.13162939  1.6924546   2.18557541]]]


 [[[ 1.19891788  0.84616065  0.82797464]
   [ 0.69803203  0.84616065  1.2245077 ]
   [ 0.69803203  1.12141771  1.2245077 ]]

  [[ 1.96710175  0.84616065  1.27375593]
   [ 1.96710175  0.84616065  1.23616403]
   [ 1.62765075  1.12141771  1.2245077 ]]

  [[ 1.96710175  0.86888616  1.27375593]
   [ 1.96710175  0.86888616  1.23616403]
   [ 1.62765075  1.12141771  0.79280687]]]]

mode = average
A.shape = (2, 3, 3, 3)
A =
 [[[[ -3.01046719e-02  -3.24021315e-03  -3.36298859e-01]
   [  1.43310483e-01   1.93146751e-01  -4.44905196e-01]
   [  1.28934436e-01   2.22428468e-01   1.25067597e-01]]

  [[ -3.81801899e-01   1.59993515e-02   1.70562706e-01]
   [  4.73707165e-02   2.59244658e-02   9.20338402e-02]
   [  3.97048605e-02   1.57189094e-01   3.45302489e-01]]

  [[ -3.82680519e-01   2.32579951e-01   6.25997903e-01]
   [ -2.47157416e-01  -3.48524998e-04   3.50539717e-01]
   [ -9.52551510e-02   2.68511000e-01   4.66056368e-01]]]


 [[[ -1.73134159e-01   3.23771981e-01  -3.43175716e-01]
   [  3.80634669e-02   7.26706274e-02  -2.30268958e-01]
   [  2.03009393e-02   1.41414785e-01  -1.23158476e-02]]

  [[  4.44976963e-01  -2.61694592e-03  -3.10403073e-01]
   [  5.08114737e-01  -2.34937338e-01  -2.39611830e-01]
   [  1.18726772e-01   1.72552294e-01  -2.21121966e-01]]

  [[  4.29449255e-01   8.44699612e-02  -2.72909051e-01]
   [  6.76351685e-01  -1.20138225e-01  -2.44076712e-01]
   [  1.50774518e-01   2.89111751e-01   1.23238536e-03]]]]
```

In [ ]:
# Case 2: stride of 2
np.random.seed(1)
A_prev = np.random.randn(2, 5, 5, 3)
hparameters = {"stride" : 2, "f": 3}

A, cache = pool_forward(A_prev, hparameters)
print("mode = max")
print("A.shape = " + str(A.shape))
print("A =\n", A)
print()

A, cache = pool_forward(A_prev, hparameters, mode = "average")
print("mode = average")
print("A.shape = " + str(A.shape))
print("A =\n", A)

**모범 답안:**
    
```
mode = max
A.shape = (2, 2, 2, 3)
A =
 [[[[ 1.74481176  0.90159072  1.65980218]
   [ 1.74481176  1.6924546   1.65980218]]

  [[ 1.13162939  1.51981682  2.18557541]
   [ 1.13162939  1.6924546   2.18557541]]]


 [[[ 1.19891788  0.84616065  0.82797464]
   [ 0.69803203  1.12141771  1.2245077 ]]

  [[ 1.96710175  0.86888616  1.27375593]
   [ 1.62765075  1.12141771  0.79280687]]]]

mode = average
A.shape = (2, 2, 2, 3)
A =
 [[[[-0.03010467 -0.00324021 -0.33629886]
   [ 0.12893444  0.22242847  0.1250676 ]]

  [[-0.38268052  0.23257995  0.6259979 ]
   [-0.09525515  0.268511    0.46605637]]]


 [[[-0.17313416  0.32377198 -0.34317572]
   [ 0.02030094  0.14141479 -0.01231585]]

  [[ 0.42944926  0.08446996 -0.27290905]
   [ 0.15077452  0.28911175  0.00123239]]]]
```

축하합니다! 이제 컨볼루션 신경망의 모든 레이어에 대한 순방향 패스를 구현했습니다.

이 과제의 나머지 부분은 선택 사항이며 채점되지 않습니다.

## 5. Backpropagation in convolutional neural networks (OPTIONAL / UNGRADED) ##

최신 딥 러닝 프레임워크에서는 정방향 연산만 구현하면 알아서 역전파를 처리해주므로 대부분의 딥러닝 엔지니어는 backward propagation의 세부 사항에 신경쓸 필요가 없습니다. 언듯 봐도 알 수 있겠지만, 컨볼루션 신경망의 오차 역전파를 계산하는 과정은 매우 복잡합니다. 따라서 여러분은 원하는 경우 이 파트를 통해 컨볼루션 신경망에서 역전파가 어떻게 이루어지는지 파악할 수 있습니다.

이전 과정에서 간단한 (Fully Connected) 신경망을 구현할 때 역전파를 사용하여 매개 변수 업데이트 비용에 대한 그래디언트를 계산했습니다. 마찬가지로, 컨볼루션 신경망에서는 매개 변수를 업데이트하기 위해 `cost`에 대한 미분을 계산할 수 있습니다. 역 전파 방정식은 사소한 것이 아니며 강의에서 도출하지 않았지만 아래에서 간략하게 설명하겠습니다.

역전파를 계산하기 위한 공식은 결코 간단하지 않으며, 강의에서도 별도로 다루지 않았지만 아래에서 간략히 설명해보겠습니다.

### 5-1. Convolutional layer backward pass ###

CONV 레이어에 대해 역전파를 를 구현해보겠습니다.

#### 5-1-1. Computing dA: ####

아래 공식은 특정 필터 $W_c$ 및 주어진 훈련 데이터 세트의 비용에 대한 미분계수 $dA$를 계산하는 공식입니다.

$$ dA += \sum _{h=0} ^{n_H} \sum_{w=0} ^{n_W} W_c \times dZ_{hw} \tag{1}$$

여기서 $W_c$는 필터이고, $dZ_{hw}$는 h번째 행과 w번째 열에서 CONV 레이어 Z의 출력에 대한 비용의 그래디언트(미분계수: 순간변화율)에 해당하는 스칼라입니다($i$ 번째 스트라이드 왼쪽 및 $j$ 번째 스트라이드에서 취한 내적에 해당). $dA$를 업데이트할 때 매번 동일한 필터 $W_c$에 다른 $dZ$를 곱합니다. 정방향 연산을 계산할 때와 마찬가지로 각 필터가 서로 다른 slice로 구분되고 곱해지기 때문입니다. 그러므로 $dA$를 계산할 때, 모든 `a_silce`에 대한 그래디언트를 합산해줍니다.


적절한 for 루프 내에서이 수식은 다음과 같이 변환됩니다.
```python
da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]
```

#### 5-1-2. Computing dW: ####


다음은 손실함수와 에 대하여 $dW_c$($dW_c$는 한 필터의 미분)를 계산하는 공식입니다.

$$ dW_c  += \sum _{h=0} ^{n_H} \sum_{w=0} ^ {n_W} a_{slice} \times dZ_{hw}  \tag{2}$$

$a_{slice}$는 활성화 변수 $Z_{ij}$를 생성하는데 사용된 슬라이스에 해당합니다. 따라서 이 값은 해당 슬라이스에 대한 $W$의 그래디언트를 제공합니다. 같은 $W$이므로 $dW$를 얻기 위해 모든 슬라이스에 대해 그래디언트를 더합니다.

#### 5-1-3. Computing db: ####

아래 공식은 특정 필터 $Wc$의 비용과 관련하여 $db$를 계산하는 공식입니다.

$$ db = \sum_h \sum_w dZ_{hw} \tag{3}$$

이전의 기본적인 인공 신경망에서도 봤듯이, $db$는 단순히 $dZ$를 더하는 것으로 구할 수 있습니다. 이번 케이스에서는 비용함수에 대한 CONV 레이어 출력 Z의 그래디언트를 모두 더하는 것으로 $db$를 구할 수 있습니다.

적절한 for 루프 내에서 코드는 아래와 같습니다.
```python
db[:,:,:,c] += dZ[i, h, w, c]
```

**연습 문제**: 아래 `conv_backward()` 함수를 구현해보세요. 모든 훈련 데이터, 필터, 높이, 너비를 더해야 합니다. 그 후 1, 2, 3번 공식을 사용해 미분계수를 구해보세요.

In [ ]:
def conv_backward(dZ, cache):
    """
    Implement the backward propagation for a convolution function
    
    Arguments:
    dZ -- gradient of the cost with respect to the output of the conv layer (Z), numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward(), output of conv_forward()
    
    Returns:
    dA_prev -- gradient of the cost with respect to the input of the conv layer (A_prev),
               numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    dW -- gradient of the cost with respect to the weights of the conv layer (W)
          numpy array of shape (f, f, n_C_prev, n_C)
    db -- gradient of the cost with respect to the biases of the conv layer (b)
          numpy array of shape (1, 1, 1, n_C)
    """
    
    ### START CODE HERE ###
    # Retrieve information from "cache"
    (A_prev, W, b, hparameters) = None
    
    # Retrieve dimensions from A_prev's shape
    (m, n_H_prev, n_W_prev, n_C_prev) = None
    
    # Retrieve dimensions from W's shape
    (f, f, n_C_prev, n_C) = None
    
    # Retrieve information from "hparameters"
    stride = None
    pad = None
    
    # Retrieve dimensions from dZ's shape
    (m, n_H, n_W, n_C) = None
    
    # Initialize dA_prev, dW, db with the correct shapes
    dA_prev = None                           
    dW = None
    db = None

    # Pad A_prev and dA_prev
    A_prev_pad = None
    dA_prev_pad = None
    
    for i in range(None):                       # loop over the training examples
        
        # select ith training example from A_prev_pad and dA_prev_pad
        a_prev_pad = None
        da_prev_pad = None
        
        for h in range(None):                   # loop over vertical axis of the output volume
            for w in range(None):               # loop over horizontal axis of the output volume
                for c in range(None):           # loop over the channels of the output volume
                    
                    # Find the corners of the current "slice"
                    vert_start = None
                    vert_end = None
                    horiz_start = None
                    horiz_end = None
                    
                    # Use the corners to define the slice from a_prev_pad
                    a_slice = None

                    # Update gradients for the window and the filter's parameters using the code formulas given above
                    da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += None
                    dW[:,:,:,c] += None
                    db[:,:,:,c] += None
                    
        # Set the ith training example's dA_prev to the unpadded da_prev_pad (Hint: use X[pad:-pad, pad:-pad, :])
        dA_prev[i, :, :, :] = None
    ### END CODE HERE ###
    
    # Making sure your output shape is correct
    assert(dA_prev.shape == (m, n_H_prev, n_W_prev, n_C_prev))
    
    return dA_prev, dW, db

In [ ]:
# We'll run conv_forward to initialize the 'Z' and 'cache_conv",
# which we'll use to test the conv_backward function
np.random.seed(1)
A_prev = np.random.randn(10,4,4,3)
W = np.random.randn(2,2,3,8)
b = np.random.randn(1,1,1,8)
hparameters = {"pad" : 2,
               "stride": 2}
Z, cache_conv = conv_forward(A_prev, W, b, hparameters)

# Test conv_backward
dA, dW, db = conv_backward(Z, cache_conv)
print("dA_mean =", np.mean(dA))
print("dW_mean =", np.mean(dW))
print("db_mean =", np.mean(db))

**모범 답안:**
<table>
    <tr>
        <td>
            <b>dA_mean</b>
        </td>
        <td>
            1.45243777754
        </td>
    </tr>
    <tr>
        <td>
            <b>dW_mean</b>
        </td>
        <td>
            1.72699145831
        </td>
    </tr>
    <tr>
        <td>
            <b>db_mean</b>
        </td>
        <td>
            7.83923256462
        </td>
    </tr>

</table>

### 5-2. Pooling layer - backward pass ###


다음으로 MAX-POOL 레이어부터 시작하여 풀링 레이어에 대한 역방향 연산을 구현해 보겠습니다. 풀링 레이어에 업데이트 할 역 전파 매개 변수가 없더라도 풀링 레이어 이전의 레이어에 대한 그래디언트를 계산하려면 풀링 레이어를 통해 그래디언트를 계산해야 합니다.

#### 5-2-1. Max pooling - backward pass ####


풀링 레이어의 역전 파로 이동하기 전에 다음을 수행하는 `create_mask_from_window()` 라는 helper 함수를 빌드합니다.

$$ X = \begin{bmatrix}
1 && 3 \\
4 && 2
\end{bmatrix} \quad \rightarrow  \quad M =\begin{bmatrix}
0 && 0 \\
1 && 0
\end{bmatrix}\tag{4}$$



보시다시피 이 함수는 행렬의 최대 값이 어디에 있는지 추적하는 "마스크"행렬을 만듭니다. True(1) 는 X의 최대 위치를 나타내고 다른 항목은 False(0) 입니다. 나중에 Average Pooling 대한 역방향 패스가 이와 비슷하지만 다른 마스크를 사용한다는 것을 알 수 있습니다.

**연습 문제**: `create_mask_from_window()` 함수를 구현해보세요. 이 함수는 pooling 역전파 연산에 도움을 줍니다.

**힌트:**
- [np.max()]()는 주어진 배열에서 최댓값을 구하는 함수입니다.
- 만약 배열 X와 스칼라 x가 있다면, `A = (X == x)` 는 X와 같은 사이즈에, 아래 조건을 만족하는 새로운 배열 A를 리턴합니다.:
```
A[i,j] = True if X[i,j] = x
A[i,j] = False if X[i,j] != x
```
- 여기에서는 행렬에 최대 값이 여러 개인 경우를 고려할 필요가 없습니다.

In [ ]:
def create_mask_from_window(x):
    """
    Creates a mask from an input matrix x, to identify the max entry of x.
    
    Arguments:
    x -- Array of shape (f, f)
    
    Returns:
    mask -- Array of the same shape as window, contains a True at the position corresponding to the max entry of x.
    """
    
    ### START CODE HERE ### (≈1 line)
    mask = None
    ### END CODE HERE ###
    
    return mask

In [ ]:
np.random.seed(1)
x = np.random.randn(2,3)
mask = create_mask_from_window(x)
print('x = ', x)
print("mask = ", mask)

**모범 답안:** 

<table> 
<tr> 
<td>
<b>x =</b>
</td>
<td>
[[ 1.62434536 -0.61175641 -0.52817175] <br>
 [-1.07296862  0.86540763 -2.3015387 ]]
  </td>
</tr>

<tr> 
<td>
<b>mask =</b>
</td>
<td>
[[ True False False] <br>
 [False False False]]
</td>
</tr>
</table>

왜 우리는 배열의 최댓값을 ​​추적할까요? 이 최댓값이 배열의 출력값에 영향을 미쳤던 입력 값이기 때문에 궁극적으로 비용에 영향을 미쳤기 때문입니다. 역전파는 비용과 관련하여 기울기를 계산하므로 궁극적으로 비용에 영향을 미치는 모든 값은 0이 아닌 기울기를 가져야합니다. 따라서 역전파는 비용에 영향을 준 이 특정 입력 값으로 기울기를 다시 "전파"합니다.

#### 5-2-2. Average pooling - backward pass ####

Max Pooling에서 하나의 입력이 출력에 줄 수 있는 "영향"은 최댓값 하나에 한정됩니다. Average Pooling에서는 입력 창의 모든 요소들이 동등하게 출력에 영향을 미칩니다. 따라서 역전파를 구현할 때, 이런 상황이 고려된 새로운 helper 함수를 만들어야 합니다.

예를 들어 2x2 필터로 average pooling을 구현했다고 하면, 역전파를 위해 사용해야할 마스크는 다음과 같이 생겼을 것입니다.

$$ dZ = 1 \quad \rightarrow  \quad dZ =\begin{bmatrix}
1/4 && 1/4 \\
1/4 && 1/4
\end{bmatrix}\tag{5}$$

이것은 $dZ$행렬의 각 위치가 정방향 연산에서 평균을 취했기 때문에, 출력의 변화에 동등하게 영향을 미친다는 것을 의미합니다.

**연습 문제**: 
아래 함수를 구현하여 주어진 차원의 행렬을 통해 값 dz를 균등하게 분배합니다. [힌트](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ones.html)

In [ ]:
def distribute_value(dz, shape):
    """
    Distributes the input value in the matrix of dimension shape
    
    Arguments:
    dz -- input scalar
    shape -- the shape (n_H, n_W) of the output matrix for which we want to distribute the value of dz
    
    Returns:
    a -- Array of size (n_H, n_W) for which we distributed the value of dz
    """
    
    ### START CODE HERE ###
    # Retrieve dimensions from shape (≈1 line)
    (n_H, n_W) = None
    
    # Compute the value to distribute on the matrix (≈1 line)
    average = None
    
    # Create a matrix where every entry is the "average" value (≈1 line)
    a = None
    ### END CODE HERE ###
    
    return a

In [ ]:
a = distribute_value(2, (2,2))
print('distributed value =', a)

**모범 답안**: 

<table> 
<tr> 
<td>
distributed_value =
</td>
<td>
[[ 0.5  0.5]
<br\> 
[ 0.5  0.5]]
</td>
</tr>
</table>

#### 5-2-3. Putting it together: Pooling backward ####


이제 풀링 레이어에서 역전파를 계산하는 데 필요한 모든 코드가 준비되었습니다.

**연습 문제**: 두 가지 모드 ( "max"및 "average")에 대해 `pool_backward` 함수를 구현해보세요. 다시 한 번 4 개의 for 루프를 사용합니다 (훈련 세트의 개수, 높이, 너비, 채널 수를 반복). 모드가 'max'또는 'average'인지 확인하려면 if / elif 문을 사용해야합니다. 'average'와 같으면 위에서 구현 한 `deploy_value()` 함수를 사용하여 `a_slice`와 같은 모양의 행렬을 만들어야합니다. 그렇지 않으면 모드는 'max'와 같고 `create_mask_from_window()`로 마스크를 만들고 dA의 해당 값을 곱합니다.

In [ ]:
def pool_backward(dA, cache, mode = "max"):
    """
    Implements the backward pass of the pooling layer
    
    Arguments:
    dA -- gradient of cost with respect to the output of the pooling layer, same shape as A
    cache -- cache output from the forward pass of the pooling layer, contains the layer's input and hparameters 
    mode -- the pooling mode you would like to use, defined as a string ("max" or "average")
    
    Returns:
    dA_prev -- gradient of cost with respect to the input of the pooling layer, same shape as A_prev
    """
    
    ### START CODE HERE ###
    
    # Retrieve information from cache (≈1 line)
    (A_prev, hparameters) = None
    
    # Retrieve hyperparameters from "hparameters" (≈2 lines)
    stride = None
    f = None
    
    # Retrieve dimensions from A_prev's shape and dA's shape (≈2 lines)
    m, n_H_prev, n_W_prev, n_C_prev = None
    m, n_H, n_W, n_C = None
    
    # Initialize dA_prev with zeros (≈1 line)
    dA_prev = None
    
    for i in range(None):                       # loop over the training examples
        
        # select training example from A_prev (≈1 line)
        a_prev = None
        
        for h in range(None):                   # loop on the vertical axis
            for w in range(None):               # loop on the horizontal axis
                for c in range(None):           # loop over the channels (depth)
                    
                    # Find the corners of the current "slice" (≈4 lines)
                    vert_start = None
                    vert_end = None
                    horiz_start = None
                    horiz_end = None
                    
                    # Compute the backward propagation in both modes.
                    if mode == "max":
                        
                        # Use the corners and "c" to define the current slice from a_prev (≈1 line)
                        a_prev_slice = None
                        # Create the mask from a_prev_slice (≈1 line)
                        mask = None
                        # Set dA_prev to be dA_prev + (the mask multiplied by the correct entry of dA) (≈1 line)
                        dA_prev[i, vert_start: vert_end, horiz_start: horiz_end, c] += None
                        
                    elif mode == "average":
                        
                        # Get the value a from dA (≈1 line)
                        da = None
                        # Define the shape of the filter as fxf (≈1 line)
                        shape = None
                        # Distribute it to get the correct slice of dA_prev. i.e. Add the distributed value of da. (≈1 line)
                        dA_prev[i, vert_start: vert_end, horiz_start: horiz_end, c] += None
                        
    ### END CODE ###
    
    # Making sure your output shape is correct
    assert(dA_prev.shape == A_prev.shape)
    
    return dA_prev

In [ ]:
np.random.seed(1)
A_prev = np.random.randn(5, 5, 3, 2)
hparameters = {"stride" : 1, "f": 2}
A, cache = pool_forward(A_prev, hparameters)
dA = np.random.randn(5, 4, 2, 2)

dA_prev = pool_backward(dA, cache, mode = "max")
print("mode = max")
print('mean of dA = ', np.mean(dA))
print('dA_prev[1,1] = ', dA_prev[1,1])  
print()
dA_prev = pool_backward(dA, cache, mode = "average")
print("mode = average")
print('mean of dA = ', np.mean(dA))
print('dA_prev[1,1] = ', dA_prev[1,1])

**모범 답안**: 

mode = max:
<table> 
<tr> 
<td>
<b>mean of dA =</b>
</td>
<td>
0.145713902729
  </td>
</tr>
<tr> 
<td>
<b>dA_prev[1,1] =</b>
</td>
<td>
[[ 0.          0.        ] <br>
 [ 5.05844394 -1.68282702] <br>
 [ 0.          0.        ]]
</td>
</tr>
</table>

mode = average
<table> 
<tr> 
<td>
<b>mean of dA =</b>
</td>
<td>
0.145713902729

  </td>
</tr>
<tr> 
<td>
<b>dA_prev[1,1] =</b>
</td>
<td>
[[ 0.08485462  0.2787552 ] <br>
 [ 1.26461098 -0.25749373] <br>
 [ 1.17975636 -0.53624893]]
</td>
</tr>
</table>


### 축하합니다 !

이 과제를 완료 한 것을 축하합니다. 이제 컨볼 루션 신경망이 작동하는 방식을 이해했습니다. 신경망의 모든 구성 요소를 구현했습니다. 다음 과제에서는 TensorFlow를 사용하여 ConvNet을 구현합니다.